In [1]:
from __future__ import division
import random
from sympy import Point, Segment
from math import cos, sin, radians
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, show
from bokeh.models import Label, ColumnDataSource, LabelSet, HoverTool, WheelZoomTool, PanTool, PolySelectTool

from mymodule.model.axis import Axis

from mymodule.model.mapper.starmapper import StarMapper
from mymodule.io.reader import Reader
from mymodule.util.axisgenerator import AxisGenerator
from mymodule.util.dfmatrixutils import DFMatrixUtils


output_notebook()

Loading BokehJS ...

In [4]:
source_file = "C:/Users/gchicafernandez/Desktop/Personal/TFGs/Data Visualization/mymodule/resources/cereal_data_set/cereal.csv"

## INTERNAL MAPPING  ##
reader = Reader.init_from_file(source_file)
dimension_labels = reader.get_dimension_labels()
dimension_values_df = reader.get_dimension_values()
axis_df = AxisGenerator.generate_star_axis(dimension_labels, random_weights=True)

vectors_df = DFMatrixUtils.get_vectors(axis_df)

mapper = StarMapper()
mapped_points = mapper.map_points(vectors_df, dimension_values_df)

## PLOT IN BOKEH ##

# Dimension related data (axis)
source_dimensions = ColumnDataSource(data=dict(x=axis_df['x1'],
                                    y=axis_df['y1'],
                                    names=axis_df.index))

labels_dimensions = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=5, y_offset=5, source=source_dimensions, render_mode='canvas')

mapped_points_x = []
mapped_points_y = []

for mpx, mpy in mapped_points:
    mapped_points_x.append(mpx)
    mapped_points_y.append(mpy)

# Point related data (products)
source_points = ColumnDataSource(data=dict(x=mapped_points_x,
                                    y=mapped_points_y,
                                    names=dimension_values_df.index))

labels_points = LabelSet(x='x', y='y', text='names', level='glyph',
              x_offset=5, y_offset=5, source=source_points, render_mode='canvas')

# Define fields that will appear when hovering over an element
hover = HoverTool(
        tooltips=[
            ("name", "@names")
        ]
    )

TOOLS = 'box_zoom,box_select,resize,reset'

p = figure(width=600, height=400, tools=[WheelZoomTool(), PanTool(), hover])

p.segment(x0=axis_df['x0'], y0=axis_df['y0'], x1=axis_df['x1'], y1=axis_df['y1'], color="#F4A582", line_width=2)
p.square('x', 'y', size=7, color="#74ADD1", alpha=0.5, source=source_dimensions)

""" Alternative:
for index, row in axis_df.iterrows():
    p.segment(x0=row['x0'], y0=row['y0'], x1=row['x1'], y1=row['y1'], color="#F4A582", line_width=2)
    p.square(x=row['x1'], y=row['y1'], size=7, color="#74ADD1", alpha=0.5)
    label = Label(x=row['x1'], y=row['y1'], x_units='screen', y_units='screen', text=index, render_mode='css',
      border_line_color='black', border_line_alpha=1.0,
      background_fill_color='white', background_fill_alpha=1.0)
"""

p.add_layout(labels_dimensions)
p.circle('x', 'y', size=5, color="navy", alpha=0.5, source=source_points)

show(p)

C:/Users/gchicafernandez/Desktop/Personal/TFGs/Data Visualization/mymodule/resources/cereal_data_set/cereal.csv
